# **Branch Splitting and Geometric Analysis**
<hr>

In [1]:
from vmtk import vmtkscripts
import numpy as np
import os

In [2]:
%run '../vmtk_functions.py'
%run '../vmtk_filenames.py'

In [3]:
# Path to save the generated files when extracting the surface
# We reccomend to separate it from the DICOM original dir. 
# Note that inside casePath, this code will create different sub-directories to store different data types
casePath       = '/home/iagolessa/Documents/unesp/doctorate/data/aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case1/'

# Define subdirs
imagesDir       = casePath+'images/'
surfacesDir     = casePath+'surfaces/'
centerlinesDir  = casePath+'centerlines/'
meshesDir       = casePath+'meshes/'
parentVesselDir = casePath+"parentVessel/"

print('Files saved to:', casePath)

Files saved to: /home/iagolessa/Documents/unesp/doctorate/data/aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case1/


<hr>

## **Branch Splitting**

The following cells demonstrates how to split a vascular segment into its constituent branches and, if it is the case, an aneurysm.

### *Centerlines Splitting*

Computing centerlines considering that the vascular model has severals inlets, by merging all the centerlines created.

In [6]:
# Get array with pacthes info
surface = readSurface(surfacesDir+surfaceOrientedFile)
capsGeoInfo = getPatchInfo(surface)

print(capsGeoInfo)

In [7]:
# Clears object outletBarycenters if exists
outletBarycenters = None
inletBarycenters = []
# Aneurysm top point to get aneurysm centerline
aneurysmTopPoint = (0.975106, -4.72867, 19.4678) # aneurysm location


# create tuples with inlet and outlet barycenters
for i in range( len(capsGeoInfo) ):
    if capsGeoInfo['PatchType'][i] == 'inlet':
        inletBarycenters.append(capsGeoInfo['Center'][i])
    else: 
        try:
            outletBarycenters += capsGeoInfo['Center'][i]
        except:
            outletBarycenters = capsGeoInfo['Center'][i]
            
print("Inlet ", inletBarycenters)
print("Outlet ", outletBarycenters)


# Computing centerlines
# This code computes centerlines for a generic number of inlets
# It does that by computing the centerlines for each source barycenter
# and then appending the resulting centerlines with 'vmtksurfaceappend'

centerlinesList = []

for source in inletBarycenters:
    # Instantiate vmtkcenterline object
    centerlines = vmtkscripts.vmtkCenterlines()
    # Surface to be used
    centerlines.Surface = surface
    # Type of seed selector: by source coordinate
    centerlines.SeedSelectorName = 'pointlist'
    centerlines.SourcePoints = source
    centerlines.TargetPoints = list(outletBarycenters+aneurysmTopPoint)
    centerlines.CheckNonManifold = 1
    centerlines.AppendEndPoints = 1
    centerlines.Execute()
    centerlinesList.append(centerlines.Centerlines)

# Provide managing exception if there is only one centerline
centerlineMain = centerlinesList[0]


# If more than one inlet, then the centerlines for 
# each inlet is appended here by a vmtk script

if len(centerlinesList) > 1:
    for centerline in centerlinesList[1:]:
        # Instantiate vmtksurfaceappend object
        centerlinesAppend = vmtkscripts.vmtkSurfaceAppend()
        centerlinesAppend.Surface = centerlineMain
        centerlinesAppend.Surface2 = centerline
        centerlinesAppend.Execute()
        # Store final centerlines to centerline main
        # therefore, centerlineMain will store the final complete centerline
        centerlineMain = centerlinesAppend.Surface
else:
    print('Only one source.')

# Centerline smoother
centerlineSmoother = vmtkscripts.vmtkCenterlineSmoothing()
centerlineSmoother.Centerlines = centerlineMain
centerlineSmoother.SurfaceOutputFileName = centerlinesDir+centerlineWithAneurysm
centerlineSmoother.Execute()
centerlineSmoother.IOWrite()

writeSurface(centerlineSmoother.Centerlines,centerlinesDir+centerlineWithAneurysm,mode='binary')
viewSurface(centerlineSmoother.Centerlines)

Inlet  [(-2.7129462e-07, -4.2361119e-08, -5.0461523e-07)]
Outlet  (-4.8139901, 7.1615324, 15.923727, 3.8962183, 0.30590865, 19.750572)
NonManifold check.
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...Only one source.
Writing VTK XML surface file.
Quit renderer


In [ ]:
# Compute Voronoi Diagram Separately for parent vessel reconstruction
voronoiDiagram = vmtkscripts.vmtkDelaunayVoronoi()

voronoiDiagram.Surface = surface
voronoiDiagram.CheckNonManifold = 1
voronoiDiagram.RemoveSubresolutionTetrahedra = 1
voronoiDiagram.VoronoiDiagramOutputFileName = surfacesDir+VoronoiDiagramFile

voronoiDiagram.Execute()
voronoiDiagram.IOWrite()

# Debug
# voronoiDiagram.PrintInputMembers()
# voronoiDiagram.PrintOutputMembers()

viewSurface(voronoiDiagram.Surface)

### *Centerline attributes*

Centerlines are defined by the position of their points. In order to define the position of a point along the centerline or to quantify the angular position of a point on the surface around the centerline, it is necessary to equip centerlines with two attributes, Abscissas and Normals.

Abscissas are easy to define: they measure the distances along a line. The point with abscissa 0 is not necessarily the first point of the centerline, but can be chosen to represent a landmark along a centerline, such that, if we have a population of models, all the abscissas will be referred to the same anatomical location. In absence of such location, abscissas can be generated relative to the starting point and can be offset to a different location at a later time.

Parallel transport reference systems: given a starting reference system (the analogous to the 0 abscissa point), the next reference system is generated by moving along the centerline of a small amount and rotating the system on the osculating plane of an amount equal to the change in the orientation of the line tangent. This operation is repeated until all points are equipped with a reference system. Since reference systems are rotated within the osculating plane no artificial torsion is introduced proceeding along the centerline.

The script that computes abscissas and parallel transport normals is vmtkcenterlineattributes. It computes abscissas and normals relative to the first point of each centerline (they can be offset to landmarks at a later time). As such, it should be called before 'vmtkbranchextractor', otherwise each tract into which centerlines are split will have an abscissa starting from 0 at the first point.

In [8]:
# Computing centerlines Frenet system
centerlineGeometry = vmtkscripts.vmtkCenterlineGeometry()
centerlineGeometry.Centerlines = centerlineSmoother.Centerlines
centerlineGeometry.LineSmoothing = 0
centerlineGeometry.Execute()

# Debug
# centerlineGeometry.PrintInputMembers()
centerlineGeometry.PrintOutputMembers()

centerlineSections = vmtkscripts.vmtkCenterlineSections()
centerlineSections.Surface = surface
centerlineSections.Centerlines = centerlineGeometry.Centerlines
# centerlineSections.CenterlinesOutputFileName = centerlinesDir+centerlineGeometryFile
centerlineSections.Execute()
# centerlineSections.IOWrite()

# Debug
# centerlineSections.PrintInputMembers()
centerlineSections.PrintOutputMembers()

# Computation of centerlines attributes
# -- Based on the parallel transport theory
centerlineAttributes = vmtkscripts.vmtkCenterlineAttributes()

centerlineAttributes.Centerlines = centerlineSections.Centerlines
# centerlineAttributes.CenterlinesOutputFileName = centerlinesDir+centerlinesAttributesFile
centerlineAttributes.Execute()
# centerlineAttributes.IOWrite()

# Debug
# centerlineAttributes.PrintInputMembers()
centerlineAttributes.PrintOutputMembers()

# Extracting branches of centerlines
# this script splits the centerline in branches
# corresponding to the different tubes formed by the Maximum Inscribed Spheres
branchExtractor = vmtkscripts.vmtkBranchExtractor()

# If centerline is read from external file
# branchExtractor.CenterlinesInputFileName = casePath+'centerlines/centerlinesAttributes.vtp'
# branchExtractor.IORead()

# If calculated right above
branchExtractor.Centerlines = centerlineAttributes.Centerlines
branchExtractor.RadiusArrayName = 'MaximumInscribedSphereRadius'
branchExtractor.CenterlinesOutputFileName = centerlinesDir+centerlineWithAneurysm

branchExtractor.Execute()
branchExtractor.IOWrite()

# Debug
# branchExtractor.PrintInputMembers()
branchExtractor.PrintOutputMembers()

viewCenterline(branchExtractor.Centerlines, branchExtractor.GroupIdsArrayName)

Output vmtkcenterlinegeometry members:
    Id = 0
    Centerlines = vtkPolyData
    LengthArrayName = Length
    CurvatureArrayName = Curvature
    TorsionArrayName = Torsion
    TortuosityArrayName = Tortuosity
Output vmtkcenterlinesections members:
    Id = 0
    CenterlineSections = vtkPolyData
    Centerlines = vtkPolyData
    CenterlineSectionAreaArrayName = CenterlineSectionArea
    CenterlineSectionMinSizeArrayName = CenterlineSectionMinSize
    CenterlineSectionMaxSizeArrayName = CenterlineSectionMaxSize
    CenterlineSectionShapeArrayName = CenterlineSectionShape
    CenterlineSectionClosedArrayName = CenterlineSectionClosed
Output vmtkcenterlineattributes members:
    Id = 0
    Centerlines = vtkPolyData
    AbscissasArrayName = Abscissas
    NormalsArrayName = ParallelTransportNormals
Writing VTK XML surface file.
Output vmtkbranchextractor members:
    Id = 0
    Centerlines = vtkPolyData
    GroupIdsArrayName = GroupIds
    CenterlineIdsArrayName = CenterlineIds
    TractI

### *Splitting the vessels and aneurysm surface*

Lets now move on to surface splitting. We now want to clip the original surface into its branches. We do it the following way using the vmtkbranchclipper script. The splitted surface is required to compute the branch sections and the branch metrics. 

In [9]:
# Branch Clipper
branchClipper = vmtkscripts.vmtkBranchClipper()
branchClipper.Surface = surface
branchClipper.Centerlines = branchExtractor.Centerlines

# Other parameters
branchClipper.RadiusArrayName   = branchExtractor.RadiusArrayName
branchClipper.GroupIdsArrayName = branchExtractor.GroupIdsArrayName
branchClipper.BlankingArrayName = branchExtractor.BlankingArrayName
branchClipper.RadiusArrayName   = branchExtractor.RadiusArrayName

# If only some patches must be computed
# branchClipper.GroupIds = [0] 
# branchClipper.ClipValue = 0.1

branchClipper.SurfaceOutputFileName = centerlinesDir+"surfaceBranchClipped.vtp"
branchClipper.CenterlinesOutputFileName = centerlinesDir+'centerlinesGeometry.vtp'
branchClipper.Execute()
branchClipper.IOWrite()

# Debug
# branchClipper.PrintInputMembers()
branchClipper.PrintOutputMembers()

Writing VTK XML surface file.
Writing VTK XML surface file.
Output vmtkbranchclipper members:
    Id = 0
    Surface = vtkPolyData
    Centerlines = vtkPolyData


In [10]:
# Visualization of the surface
surfaceViewer = vmtkscripts.vmtkSurfaceViewer()
surfaceViewer.Surface = branchClipper.Surface
surfaceViewer.ArrayName = branchClipper.GroupIdsArrayName
surfaceViewer.Legend = 1
surfaceViewer.Execute()

Quit renderer


### *Longitudinal and circumferential metrics*
By means of the vmtkbranchmetrics script two additional arrays are created on each branch of the split surface whose default names are AbscissaMetric and AngularMetric: the first is computed from the curvilinear abscissa defined on the centerlines, while the second represents the periodic circumferential coordinate of mesh points around the centerlines and spans the interval $(-\pi, +\pi)$.

In [5]:
branchMetrics = vmtkscripts.vmtkBranchMetrics()

branchMetrics.Surface = branchClipper.Surface
branchMetrics.Centerlines = branchExtractor.Centerlines
# branchMetrics.SurfaceInputFileName = centerlinesDir+'surfaceBranchClipped.vtp'
# branchMetrics.CenterlinesInputFileName = centerlinesDir+'centerlinesGeometry.vtp'
# branchMetrics.IORead()

branchMetrics.AbscissasArrayName = centerlineAttributes.AbscissasArrayName # 'Abscissas'
branchMetrics.NormalsArrayName   = centerlineAttributes.NormalsArrayName
branchMetrics.CenterlineIdsArrayName = branchExtractor.CenterlineIdsArrayName # 'CenterlineIds'
branchMetrics.GroupIdsArrayName = branchExtractor.GroupIdsArrayName           # 'GroupIds'
branchMetrics.TractIdsArrayName = branchExtractor.TractIdsArrayName           # 'TractIds'
branchMetrics.BlankingArrayName = branchExtractor.BlankingArrayName           # 'Blanking'
branchMetrics.RadiusArrayName   = branchExtractor.RadiusArrayName             # 'MaximumInscribedSphereRadius'

branchMetrics.SurfaceOutputFileName = centerlinesDir+'surfaceClippedMetrics.vtp'

branchMetrics.PrintInputMembers()
branchMetrics.PrintOutputMembers()

branchMetrics.Execute()
branchMetrics.IOWrite()

Reading VTK XML surface file.
Reading VTK XML surface file.
Input vmtkbranchmetrics members:
    Id = 0
    Disabled = 0
    Surface = vtkPolyData
    SurfaceInputFileName = /home/iagolessa/Documents/unesp/doctorate/data/aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case1/centerlines/surfaceBranchClipped.vtp
    Centerlines = vtkPolyData
    CenterlinesInputFileName = /home/iagolessa/Documents/unesp/doctorate/data/aneurysms/geometries/einsteinCases/rupturedCases/vmtkReconstruction/case1/centerlines/centerlinesGeometry.vtp
    ComputeAbscissaMetric = 1
    ComputeAngularMetric = 1
    AbscissasArrayName = Abscissas
    NormalsArrayName = ParallelTransportNormals
    GroupIdsArrayName = GroupIds
    CenterlineIdsArrayName = CenterlineIds
    TractIdsArrayName = TractIds
    RadiusArrayName = MaximumInscribedSphereRadius
    BlankingArrayName = Blanking
    AngularMetricArrayName = AngularMetric
    AbscissaMetricArrayName = AbscissaMetric
    SurfaceOutputFileName =

### *Computing branch sections surface*

In [12]:
branchSections = vmtkscripts.vmtkBranchSections()

branchSections.Surface = branchClipper.Surface
branchSections.Centerlines = branchExtractor.Centerlines
# branchSections.SurfaceInputFileName = centerlinesDir+'surfaceBranchClipped.vtp'
# branchSections.CenterlinesInputFileName = centerlinesDir+'centerlinesGeometry.vtp'
# branchSections.IORead()

# From centerlines attributes
branchSections.AbscissasArrayName = centerlineAttributes.AbscissasArrayName # 'Abscissas'
branchSections.NormalsArrayName   = centerlineAttributes.NormalsArrayName   # 'ParallelTransportNormals'

# From centerlines branch extractor
branchSections.CenterlineIdsArrayName = branchExtractor.CenterlineIdsArrayName # 'CenterlineIds'
branchSections.GroupIdsArrayName = branchExtractor.GroupIdsArrayName           # 'GroupIds'
branchSections.TractIdsArrayName = branchExtractor.TractIdsArrayName           # 'TractIds'
branchSections.BlankingArrayName = branchExtractor.BlankingArrayName           # 'Blanking'
branchSections.RadiusArrayName   = branchExtractor.RadiusArrayName             # 'MaximumInscribedSphereRadius'

branchSections.ReverseDirection = 0
branchSections.NumberOfDistanceSpheres = 1

branchSections.BranchSectionsOutputFileName = centerlinesDir+"surfaceBranchSections.vtp"
branchSections.Execute()
branchSections.IOWrite()


# Debug
branchSections.PrintInputMembers()
branchSections.PrintOutputMembers()

Writing VTK XML surface file.
Input vmtkbranchsections members:
    Id = 0
    Disabled = 0
    Surface = vtkPolyData
    SurfaceInputFileName = 
    Centerlines = vtkPolyData
    CenterlinesInputFileName = 
    NumberOfDistanceSpheres = 1
    ReverseDirection = 0
    RadiusArrayName = MaximumInscribedSphereRadius
    GroupIdsArrayName = GroupIds
    CenterlineIdsArrayName = CenterlineIds
    TractIdsArrayName = TractIds
    BlankingArrayName = Blanking
    BranchSectionGroupIdsArrayName = BranchSectionGroupIds
    BranchSectionAreaArrayName = BranchSectionArea
    BranchSectionMinSizeArrayName = BranchSectionMinSize
    BranchSectionMaxSizeArrayName = BranchSectionMaxSize
    BranchSectionShapeArrayName = BranchSectionShape
    BranchSectionClosedArrayName = BranchSectionClosed
    BranchSectionDistanceSpheresArrayName = BranchSectionDistanceSpheres
    BranchSectionsOutputFileName = /home/iagolessa/Documents/unesp/doctorate/data/aneurysms/geometries/einsteinCases/rupturedCases/vmtkRe

# Metrics mapping to branches
By construction of a harmonic function (Figure 4-left) over each vascular segment, vmtkbranchmapping maps and stretches the longitudinal metric to correctly account for the presence of insertion regions at bifurcations; the additional StretchedMapping array is added to the surface (Figure 4-middle).

In [6]:
branchMapping = vmtkscripts.vmtkBranchMapping()

# Read surface or use the calculated above
# branchMapping.SurfaceInputFileName = casePath+'surfaces/branchClipped.vtp'
branchMapping.Surface = branchMetrics.Surface
branchMapping.CenterlinesInputFileName = casePath+'centerlines/branchExtracted.vtp'
branchMapping.ReferenceSystemsInputFileName = casePath+'centerlines/bifurcationRefSystem.vtp'
branchMapping.IORead()

branchMapping.AbscissasArrayName = 'Abscissas'
branchMapping.NormalsArrayName = 'ParallelTransportNormals'
branchMapping.CenterlineIdsArrayName = 'CenterlineIds'
branchMapping.GroupIdsArrayName = 'GroupIds'
branchMapping.TractIdsArrayName = 'TractIds'
branchMapping.BlankingArrayName = 'Blanking'
branchMapping.RadiusArrayName = 'MaximumInscribedSphereRadius'

branchMapping.AngularMetricArrayName = 'AngularMetric' # from cell above (branch metrics)
branchMapping.AbscissaMetricArrayName = 'AbscissaMetric' # from cell above (branch metrics)

branchMapping.SurfaceOutputFileName = casePath+'branchAnalysis/surfaceClippedMapping.vtp'

branchMapping.PrintInputMembers()
branchMapping.PrintOutputMembers()

branchMapping.Execute()
branchMapping.IOWrite()

Reading VTK XML surface file.

Reading VTK XML surface file.

Input vmtkbranchmapping members:

    Id = 0

    Disabled = 0

    Surface = vtkPolyData

    SurfaceInputFileName = 

    Centerlines = vtkPolyData

    CenterlinesInputFileName = /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/vmtkReconstruction/71_MLS/centerlines/branchExtracted.vtp

    ReferenceSystems = vtkPolyData

    ReferenceSystemsInputFileName = /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/vmtkReconstruction/71_MLS/centerlines/bifurcationRefSystem.vtp

    AbscissasArrayName = Abscissas

    NormalsArrayName = ParallelTransportNormals

    GroupIdsArrayName = GroupIds

    CenterlineIdsArrayName = CenterlineIds

    TractIdsArrayName = TractIds

    ReferenceSystemsNormalArrayName = 

    RadiusArrayName = MaximumInscribedSphereRadius

    BlankingArrayName = Blanking

    AngularMetricArrayName = AngularMetric

    HarmonicMappingArrayName = HarmonicMapping

    

In [12]:
branchGeometry = vmtkscripts.vmtkBranchGeometry()

branchGeometry.PrintInputMembers()
branchGeometry.PrintOutputMembers()

Input vmtkbranchgeometry members:

    Id = 0

    Disabled = 0

    Centerlines = None

    CenterlinesInputFileName = 

    RadiusArrayName = 

    GroupIdsArrayName = 

    BlankingArrayName = 

    LengthArrayName = Length

    CurvatureArrayName = Curvature

    TorsionArrayName = Torsion

    TortuosityArrayName = Tortuosity

    LineSmoothing = 0

    NumberOfSmoothingIterations = 100

    SmoothingFactor = 0.1

    GeometryDataOutputFileName = 

Output vmtkbranchgeometry members:

    Id = 0

    GeometryData = None

    LengthArrayName = Length

    CurvatureArrayName = Curvature

    TorsionArrayName = Torsion

    TortuosityArrayName = Tortuosity


<hr>

## *Patching of surface mesh and attributes*

All the ingredients are now in place to perform the real patching of the surface, that is to "cut" a set of contiguous rectangular regions on the mesh that follow iso-contours in the StretchedMapping and AngularMetric arrays; all the quantities of interest (WSS and OSI in this case) are averaged over these areas.

By means of the options -longitudinalpatchsize and -circularpatches we impose the dimensions of the patches, in terms of height (in mm) of the patch along the longitudinal direction and number of angular cut over the interval $(-\pi, +\pi)$ respectively; the result of this discretization can be seen visualizing the Slab and Sector arrays created by the script or the mesh new surface discretization (Figure 5). You will probably have to play with these values to find out the discretization that best fits your needs; we here cut every 0.5 mm and subdivide the interval $(-\pi, +\pi)$ in 12 sectors.

In [17]:
branchPatching = vmtkscripts.vmtkBranchPatching()

branchPatching.SurfaceInputFileName = casePath+'branchAnalysis/surfaceClippedMapping.vtp'
branchPatching.IORead()
branchPatching.GroupIdsArrayName = 'GroupIds'
branchPatching.LongitudinalMappingArrayName = 'StretchedMapping'
branchPatching.CircularMappingArrayName = 'AngularMetric'
branchPatching.LongitudinalPatchSize = 0.5
branchPatching.CircularNumberOfPatches = 12

branchPatching.SurfaceOutputFileName = casePath+'branchAnalysis/surfaceClippedPatching.vtp'

branchPatching.PrintInputMembers()
branchPatching.PrintOutputMembers()

branchPatching.Execute()
branchPatching.IOWrite()

Reading VTK XML surface file.

Input vmtkbranchpatching members:

    Id = 0

    Disabled = 0

    Surface = vtkPolyData

    SurfaceInputFileName = /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/vmtkReconstruction/38_RSS/branchAnalysis/surfaceClippedMapping.vtp

    PatchSize = [0.0, 0.0]

    LongitudinalPatchSize = 0.5

    CircularNumberOfPatches = 12

    CircularPatching = 1

    UseConnectivity = 1

    GroupIdsArrayName = GroupIds

    LongitudinalMappingArrayName = StretchedMapping

    CircularMappingArrayName = AngularMetric

    LongitudinalPatchNumberArrayName = Slab

    CircularPatchNumberArrayName = Sector

    PatchAreaArrayName = PatchArea

    SurfaceOutputFileName = /home/iagolessa/documents/unesp/doctorate/aneurysms/geometries/vrCases/vmtkReconstruction/38_RSS/branchAnalysis/surfaceClippedPatching.vtp

    PatchedDataOutputFileName = 

Output vmtkbranchpatching members:

    Id = 0

    Surface = vtkPolyData

    PatchedData = None

    Pat

<hr>

# **Geometrical Analysis**

This tutorial demonstrates how to analyze the 3D geometry of a vascular segment. Using vmtk, you should have been able to compute centerlines, to identify bifurcations and branches on the centerlines and to clip the surface according to the branches. Now we will face the problem of quantifying geometric features of the vascular segment, those associated to bifurcations, such as bifurcation planes and bifurcation angles, and those associated to branches, such as curvature and torsion.

## Computing bifurcation reference systems 

We here consider a single bifurcation: the extension to a more general case is rather straightforward. From the centerline splitting and grouping phase we were left with a first group that ends with the second reference points (one for each tract), a second group representing the bifurcation whose tracts start with the second reference point and end with the first, and two last groups representing the daughter branches that start with the first reference points (where each centerline exits the other tube). We define the bifurcation reference system the following way:

--> The origin of the bifurcation is defined as the barycenter of the four reference points weighted by the surface of the maximum inscribed sphere defined on the reference points. The reason of the weighting is that small branches have less impact on the position of the bifurcation origin;

--> The normal to the bifurcation plane is defined as the normal to the polygon defined by the four reference points computed in the bifurcation origin;

--> The bifurcation upnormal is the similarly weighted average vector between the vectors pointing from the second to the first reference point on each centerline

In [ ]:
# Computing the bifurcation reference system
# it uses the output of Branch Extractor script because 
# it depends on array calculated there, especially 
# because it identifies the bifurcations and the tracts
bifurcationRefSystem = vmtkscripts.vmtkBifurcationReferenceSystems()

# inputFile = casePath+"centerlines/centerlines.vtp"
# bifurcationRefSystem.CenterlinesInputFileName = inputFile
# bifurcationRefSystem.IORead()

bifurcationRefSystem.Centerlines = branchExtractor.Centerlines
bifurcationRefSystem.RadiusArrayName = branchExtractor.RadiusArrayName
bifurcationRefSystem.BlankingArrayName = branchExtractor.BlankingArrayName
bifurcationRefSystem.GroupIdsArrayName = branchExtractor.GroupIdsArrayName

bifurcationRefSystem.ReferenceSystemsOutputFileName = centerlinesDir+'bifurcationRefSystem.vtp'

bifurcationRefSystem.Execute()
# Writing to disk parameters
bifurcationRefSystem.IOWrite()

# Debug
# bifurcationRefSystem.PrintInputMembers()
bifurcationRefSystem.PrintOutputMembers()

## Offsetting centerline attributes to bifurcations

As anticipated, centerline attributes (i.e. abscissas and parallel transport normals), which were computed relative to the first point of the centerline (not a very meaningful landmark), can be offset to bifurcations. This way, a population of vessels will have the 0 abscissa corresponding to the origin of the same bifurcation, which sounds like a reasonable thing to do. Similarly, parallel transport normals at the bifurcation will be oriented like the bifurcation plane normal.

In [ ]:
# To offset the centerline attributes, we need to chose the bifurcation
# so inspecting centerlines

vmtk_functions.viewCenterline(branchExtractor.Centerlines, branchExtractor.GroupIdsArrayName)

In [ ]:
bifurcationId = 1

# Offsetting the centerlines attributes
centerlineOffsetAttributes = vmtkscripts.vmtkCenterlineOffsetAttributes()

# Input parameters 
centerlineOffsetAttributes.Centerlines = branchExtractor.Centerlines
centerlineOffsetAttributes.ReferenceSystems = bifurcationRefSystem.ReferenceSystems

centerlineOffsetAttributes.AbscissasArrayName = centerlineAttributes.AbscissasArrayName
centerlineOffsetAttributes.NormalsArrayName   = centerlineAttributes.NormalsArrayName
centerlineOffsetAttributes.GroupIdsArrayName  = branchExtractor.GroupIdsArrayName
centerlineOffsetAttributes.CenterlineIdsArrayName = branchExtractor.CenterlineIdsArrayName

normalsArrayName = bifurcationRefSystem.ReferenceSystemsNormalArrayName
centerlineOffsetAttributes.ReferenceSystemsNormalArrayName = normalsArrayName

# Bifurcation ID
centerlineOffsetAttributes.ReferenceGroupId = bifurcationId

# Writing parameters
# centerlineOffsetAttributes.CenterlinesOutputFileName = centerlinesDir+'bifurcationRefSystemOffset.vtp'

centerlineOffsetAttributes.Execute()

# Writing to disk parameters
# centerlineOffsetAttributes.IOWrite()

# Debug
# centerlineOffsetAttributes.PrintInputMembers()
centerlineOffsetAttributes.PrintOutputMembers()

### *Computing bifurcation geometry*

Each branch, parent or daughter, arrives or leaves the bifurcation region with a certain orientation. Such orientation can in general be decomposed in in-plane and out-of-plane components, where the plane is the bifurcation plane. The geometry of the bifurcation will therefore be described by the in-plane and out-of-plane angles that each branch forms with the bifurcation reference system. Taking the upnormal as an in-plane reference orientation, each branch at the bifurcation is described by its in-plane angle with the upnormal direction and by its out-of-plane angle with the bifurcation normal.

In [ ]:
# To compute bifurcation geometry we use the vmtkbifurcationvectors
bifurcationVectors = vmtkscripts.vmtkBifurcationVectors()

# Input parameters
bifurcationVectors.Centerlines = branchExtractor.Centerlines
 
bifurcationVectors.ReferenceSystems = bifurcationRefSystem.ReferenceSystems
bifurcationVectors.RadiusArrayName = branchExtractor.RadiusArrayName
bifurcationVectors.CenterlineIdsArrayName = branchExtractor.CenterlineIdsArrayName
bifurcationVectors.GroupIdsArrayName = branchExtractor.GroupIdsArrayName
bifurcationVectors.TractIdsArrayName = branchExtractor.TractIdsArrayName
bifurcationVectors.BlankingArrayName = branchExtractor.BlankingArrayName
bifurcationVectors.ReferenceSystemsNormalArrayName = bifurcationRefSystem.ReferenceSystemsNormalArrayName
bifurcationVectors.ReferenceSystemsUpNormalArrayName = bifurcationRefSystem.ReferenceSystemsUpNormalArrayName
# bifurcationVectors.BifurcationVectorsOutputFileName = centerlinesDir+'bifurcationVectors.vtp' 

bifurcationVectors.Execute()
# bifurcationVectors.IOWrite()

# Debug
# bifurcationVectors.PrintInputMembers()
bifurcationVectors.PrintOutputMembers()

In [ ]:
# Computing bifurcations sections 
bifurcationSections =  vmtkscripts.vmtkBifurcationSections()

# Surface and centerlines already split into branches
bifurcationSections.Surface= branchClipper.Surface
bifurcationSections.Centerlines = branchClipper.Centerlines
bifurcationSections.NumberOfDistanceSpheres = 0

# Arrays specifications
bifurcationSections.RadiusArrayName = branchExtractor.RadiusArrayName
bifurcationSections.CenterlineIdsArrayName = branchExtractor.CenterlineIdsArrayName
bifurcationSections.GroupIdsArrayName = branchExtractor.GroupIdsArrayName
bifurcationSections.TractIdsArrayName = branchExtractor.TractIdsArrayName
bifurcationSections.BlankingArrayName = branchExtractor.BlankingArrayName
bifurcationSections.BifurcationSectionsOutputFileName = centerlinesDir+'bifurcationSections0.vtp'

bifurcationSections.Execute()
bifurcationSections.IOWrite()

# Debug
# bifurcationSections.PrintInputMembers()
bifurcationSections.PrintOutputMembers()

In [ ]:
# Computing bifurcations profiles (curves on branch extractor joining locations)
bifurcationProfiles =  vmtkscripts.vmtkBifurcationProfiles()

# Surface and centerlines already split into branches
bifurcationProfiles.Surface = branchClipper.Surface
bifurcationProfiles.Centerlines = branchClipper.Centerlines

# Arrays specifications
bifurcationProfiles.RadiusArrayName = branchExtractor.RadiusArrayName
bifurcationProfiles.CenterlineIdsArrayName = branchExtractor.CenterlineIdsArrayName
bifurcationProfiles.GroupIdsArrayName = branchExtractor.GroupIdsArrayName
bifurcationProfiles.TractIdsArrayName = branchExtractor.TractIdsArrayName
bifurcationProfiles.BlankingArrayName = branchExtractor.BlankingArrayName

bifurcationProfiles.BifurcationProfilesOutputFileName = casePath+"centerlines/bifurcationProfiles.vtp"

bifurcationProfiles.Execute()
bifurcationProfiles.IOWrite()

# Debug
# bifurcationProfiles.PrintInputMembers()
bifurcationProfiles.PrintOutputMembers()